In [ ]:
# https://github.com/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb
# https://www.youtube.com/watch?v=hv3A62Ojqdg&t=418s

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 5.7MB/s 
     |████████████████████████████████| 112kB 21.9MB/s 
     |████████████████████████████████| 26.9MB 78.4MB/s 


In [3]:
!nvidia-smi

Thu Mar 18 18:09:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## mount google drive and install repo

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
if os.path.isdir("/content/drive/My Drive/GAN/colab-sg2-ada"):
    %cd "/content/drive/My Drive/GAN/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/GAN/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/GAN/colab-sg2-ada/stylegan2-ada


In [6]:
import os
if os.path.isdir("/content/drive/My Drive/GAN/stylegan2-ada-pytorch"):
    %cd "/content/drive/My Drive/GAN/stylegan2-ada-pytorch"
else:
    #install script
    %cd "/content/drive/My Drive/GAN/"
    !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

/content/drive/My Drive/GAN/stylegan2-ada-pytorch


In [12]:
!ls

calc_metrics.py  Dockerfile	generate.py  projector.py     training
data		 docker_run.sh	legacy.py    __pycache__      training-runs
datasets	 docs		LICENSE.txt  README.md	      train.py
dataset_tool.py  downloads	metrics      style_mixing.py
dnnlib		 download.sh	out	     torch_utils


In [28]:
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl"...
Generating image for seed 85 (0/4) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 265 (1/4) ...
Generating image for seed 297 (2/4) ...
Generating image for seed 849 (3/4) ...


In [ ]:
# download the afhq dataset, download.sh is from https://github.com/clovaai/stargan-v2.git
!bash download.sh afhq-dataset

##preparing dataset

In [29]:
# preparing dataset
!python dataset_tool.py --source='data/afhq/train/cat' --dest='datasets/afhqcat.zip'

100% 5153/5153 [02:12<00:00, 38.89it/s]


##preparing custom dataset

In [ ]:
!python dataset_tool.py --help

In [ ]:
# upload cow.zip in drive
!unzip '/content/cow.zip' -d 'data/cow256'

In [24]:
import os
dir = os.listdir('data/cow')
for img in dir:
  if img.find('.png')==0:
    print(img)
print(len(dir))

769


In [26]:
!python dataset_tool.py --source='data/cow' --dest='datasets/cow256.zip'
!python dataset_tool.py --source='data/cow' --dest='datasets/cow512.zip' --width 512 --height 512

100% 769/769 [00:20<00:00, 38.31it/s]


##Training

In [10]:
!python train.py --outdir='training-runs' --data='datasets/afhqcat.zip' \
      --gpus=1 --cfg=paper512 --kimg=5


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "datasets/afhqcat.zip",
    "use_labels": false,
    "max_size": 5153,
    "xflip": false,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
 

##Training cow dataset

In [ ]:
!python train.py --help

In [ ]:
!python train.py --outdir='training-runs' --data='datasets/cow256.zip' \
      --gpus=1 --cfg=paper256 --kimg=1000 --snap=2


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 2,
  "network_snapshot_ticks": 2,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "datasets/cow256.zip",
    "use_labels": false,
    "max_size": 769,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 16384,
    "channel_max": 512,
    "